In [1]:
import os
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.store.postgres import PostgresStore
from dotenv import load_dotenv
from utils import get_llm
from langmem import create_memory_manager, create_memory_store_manager
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [2]:
from pydantic import BaseModel

class PreferenceMemory(BaseModel):
    category: str
    preference: str
    context: str

In [3]:
model = get_llm()

In [8]:
manager = create_memory_manager(
    model,
    schemas=[PreferenceMemory],
    enable_inserts=True
)

In [ ]:
# conversations_1 = [
#     HumanMessage("Alice likes dark mode in all of her apps"),
#     AIMessage("I now remember Alices preferences")
# ]

In [9]:
conversations_1 = [
            {"role": "user", "content": "Alice likes dark mode in all of her apps"},
            {"role": "assistant", "content": "I'll remember that preference"},
        ]

In [10]:
memories = await manager(conversations_1)

In [11]:
memories[0][1]

PreferenceMemory(category='appearance', preference='dark mode', context='Alice likes dark mode in all of her apps')

In [12]:
conversations_2 = [
            {"role": "user", "content": "Alice lives in Hyderabad"},
            {"role": "assistant", "content": "I'll remember that preference"},
        ]

In [13]:
memories = await manager(conversations_2)


In [14]:
for memory in memories:
    print(memory[1])

category='people_info' preference='Alice lives in Hyderabad' context='session_74c8f634-9b60-4883-92e4-d6a978b1107b'


In [15]:
conversations_3 = [
            {"role": "user", "content": "Bob lives in Hyderabad"},
            {"role": "user", "content": "Bob likes Biryani"},
            {"role": "assistant", "content": "I'll remember that preference"},
        ]

memories = await manager(conversations_3)

In [16]:
memories

[ExtractedMemory(id='8f058447-3e42-4d09-93a5-5fde473370e1', content=PreferenceMemory(category='Location', preference='Hyderabad', context="Bob's place of residence")),
 ExtractedMemory(id='a98c46b9-2a42-40c4-816e-90499dfb6753', content=PreferenceMemory(category='Food Preference', preference='Biryani', context="Bob's liking for Biryani"))]

In [2]:
from pydantic import BaseModel
class UserProfile(BaseModel):
    name: str
    city: str
    favorite_drink: str

In [3]:
model = get_llm()

In [4]:
load_dotenv()
DB_URI = os.getenv('PG_VECTOR_DB_URI')

In [5]:
from langchain_google_vertexai.embeddings import VertexAIEmbeddings

def get_embedding(name:str = 'text-embedding-005') -> VertexAIEmbeddings:
    return VertexAIEmbeddings(model=name)

def ns(user_id:str):
    return ("user", "{user_id}", "profile")

In [12]:
user_id='alice'

with PostgresStore.from_conn_string(
    DB_URI, 
    index={
            "dims": 768,
                "embed": get_embedding(),
                "fields": ["memory_text"]  # specify which fields to embed. Default is the whole serialized value
            }
    ) as store:
    store.setup()
    memory_manager = create_memory_store_manager(
        model,
        schemas = [UserProfile],
        store=store,
        namespace=("user", "{user_id}", "profile")

    )

    #memories = await memory_manager(conversations_3)
    response = memory_manager.invoke({
        "messages": [
            {"role": "user", "content": "Alice lives in Hyderabad"},
            {"role": "user", "content": "Alice likes coffee"},
            {"role": "assistant", "content": "I remember the preferences"},
        ]
    }, {
        "configurable": {
            "thread_id": "t1",
            "user_id": "alice"
        }
    })


In [13]:
user_id='bob'

with PostgresStore.from_conn_string(
    DB_URI, 
    index={
            "dims": 768,
                "embed": get_embedding(),
                "fields": ["memory_text"]  # specify which fields to embed. Default is the whole serialized value
            }
    ) as store:
    store.setup()
    memory_manager = create_memory_store_manager(
        model,
        schemas = [UserProfile],
        store=store,
        namespace=("user", "{user_id}", "profile")

    )
    

    #memories = await memory_manager(conversations_3)
    response = memory_manager.invoke({
        "messages": [
            {"role": "user", "content": "Bob lives in Bangalore"},
            {"role": "user", "content": "Bob likes tea"},
            {"role": "assistant", "content": "I remember the preferences"},
        ]
    }, {
        "configurable": {
            "thread_id": "t2",
            "user_id": "bob"
        }
    })



In [14]:
user_id='alice'

with PostgresStore.from_conn_string(
    DB_URI, 
    index={
            "dims": 768,
                "embed": get_embedding(),
                "fields": ["memory_text"]  # specify which fields to embed. Default is the whole serialized value
            }
    ) as store:
    store.setup()
    memory_manager = create_memory_store_manager(
        model,
        schemas = [UserProfile],
        store=store,
        namespace=("user", "{user_id}", "profile")

    )

    #memories = await memory_manager(conversations_3)
    response = memory_manager.invoke({
        "messages": [
            {"role": "user", "content": "Alice relocated to Mumbai"},
            {"role": "assistant", "content": "I remember the preferences"},
        ]
    }, {
        "configurable": {
            "thread_id": "t3",
            "user_id": "alice"
        }
    })


In [ ]:
from langgraph.checkpoint.postgres import PostgresSaver

user_id='alice'

with PostgresStore.from_conn_string(
    DB_URI, 
    index={
            "dims": 768,
                "embed": get_embedding(),
                "fields": ["memory_text"]  # specify which fields to embed. Default is the whole serialized value
            }
    ) as store:
    store.setup()
    memory_manager = create_memory_store_manager(
        model,
        schemas = [UserProfile],
        store=store,
        namespace=("user", "{user_id}", "profile")

    )

    #memories = await memory_manager(conversations_3)
    response = memory_manager.invoke({
        "messages": [
            {"role": "user", "content": "Alice relocated to Mumbai"},
            {"role": "assistant", "content": "I remember the preferences"},
        ]
    }, {
        "configurable": {
            "thread_id": "t3",
            "user_id": "alice"
        }
    })
